<a href="https://colab.research.google.com/github/subhant5/VPN/blob/main/NLP_Language_Modelling_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.utils import np_utils as ku 
import numpy as np 

### **Text Preprocessing and sequence generation**

In this step we will create a function which will take the training text data and perform all the text preprocessing tasks. After preprocessing we will convert the data into labels and predictors which will be returned by the function.

In [ ]:
tokenizer = Tokenizer()

def dataset_preparation(data):

	# basic cleanup
	corpus = data.lower().split("\n")

	# tokenization	
	tokenizer.fit_on_texts(corpus)
	total_words = len(tokenizer.word_index) + 1

	# create input sequences using list of tokens
	input_sequences = []
	for line in corpus:
		token_list = tokenizer.texts_to_sequences([line])[0]
		for i in range(1, len(token_list)):
			n_gram_sequence = token_list[:i+1]
			input_sequences.append(n_gram_sequence)

	# pad sequences 
	max_sequence_len = max([len(x) for x in input_sequences])
	input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

	# create predictors and label
	predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
	label = ku.to_categorical(label, num_classes=total_words)

	return predictors, label, max_sequence_len, total_words

### **Model Creation and Compilation**

In this step we will create a function which will define the lstm model we will use, compile the model and fit the model on training data.

In [ ]:
def create_model(predictors, label, max_sequence_len, total_words):
	
	model = Sequential()
	model.add(Embedding(total_words, 10, input_length=max_sequence_len-1))
	model.add(LSTM(150, return_sequences = True))
	model.add(Dropout(0.2))
	model.add(LSTM(100))
	model.add(Dense(total_words, activation='softmax'))

	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
	model.fit(predictors, label, epochs=500, verbose=1, callbacks=[earlystop])
	model.summary()
	return model 

### **Text Generation**
In this step we will write a function to generate text given some initial words.

In [ ]:
def generate_text(seed_text, next_words, max_sequence_len, model):
	for _ in range(next_words):
		token_list = tokenizer.texts_to_sequences([seed_text])[0]
		token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
		predicted = model.predict(token_list)
		predicted = np.argmax(predicted,-1)
		output_word = ""
		for word, index in tokenizer.word_index.items():
			if index == predicted:
				output_word = word
				break
		seed_text += " " + output_word
	return seed_text


### **Calling all the functions**

In this step we will call all the functions we made above one by one

In [ ]:
data = open('data.txt').read()

predictors, label, max_sequence_len, total_words = dataset_preparation(data)
model = create_model(predictors, label, max_sequence_len, total_words)
print(generate_text("O Mother", 3, max_sequence_len,model))

Epoch 1/500
2/2 [==============================] - 3s 22ms/step - loss: 3.7621 - accuracy: 0.0208
Epoch 2/500
2/2 [==============================] - 0s 23ms/step - loss: 3.7590 - accuracy: 0.0417
Epoch 3/500
2/2 [==============================] - 0s 19ms/step - loss: 3.7560 - accuracy: 0.0625
Epoch 4/500
2/2 [==============================] - 0s 19ms/step - loss: 3.7529 - accuracy: 0.0625
Epoch 5/500
2/2 [==============================] - 0s 18ms/step - loss: 3.7497 - accuracy: 0.0625
Epoch 6/500
2/2 [==============================] - 0s 20ms/step - loss: 3.7455 - accuracy: 0.0625
Epoch 7/500
2/2 [==============================] - 0s 20ms/step - loss: 3.7384 - accuracy: 0.0625
Epoch 8/500
2/2 [==============================] - 0s 19ms/step - loss: 3.7318 - accuracy: 0.0625
Epoch 9/500
2/2 [==============================] - 0s 20ms/step - loss: 3.7210 - accuracy: 0.0625
Epoch 10/500
2/2 [==============================] - 0s 20ms/step - loss: 3.7081 - accuracy: 0.0625
Epoch 11/500
2/2 [=

In [ ]:
print(label)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]]
